In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Libraries imported.


##### Define Foursquare Credentials and Version

In [117]:
# Pull API keys from the file
with open('4square_keys3.txt','r') as f:
    CLIENT_ID, CLIENT_SECRET = [l.strip() for l in f.readlines()]

VERSION = '20180604'
LIMIT = 500

#### Explore venues around my work place 

In [6]:
address = 'Russia, Taganrog'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, ',', longitude)

47.2153657 , 38.9285216


In [53]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            500, 
            LIMIT)

In [33]:
import json, codecs
def json_to_file(data, name='work_nbrs.json'):
    with open(name, 'wb') as f:
        json.dump(data, codecs.getwriter('utf-8')(f), ensure_ascii=False)

In [54]:
def searchVenuesInRect(sw, ne):
    
    venues_list=[]
    
    def get_category(cat_list):
        if len(cat_list) == 0:
            return None
        else:
            return cat_list[0]['name']
              
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&intent=browse&sw={},{}&ne={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            sw[0], 
            sw[1], 
            ne[0], 
            ne[1],
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()['response']['venues']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['name'], 
        v['location']['lat'], 
        v['location']['lng'],  
        get_category(v['categories']),
        v['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Venue', 'Latitude', 'Longitude', 'Category', 'Id']
    
    return(nearby_venues)
        

In [55]:
def searchVenuesInCircle(c, r):
    
    venues_list=[]
    
    def get_category(cat_list):
        if len(cat_list) == 0:
            return None
        else:
            return cat_list[0]['name']
              
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            c[0], 
            c[1], 
            r,             
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()['response']['venues']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['name'], 
        v['location']['lat'], 
        v['location']['lng'],  
        get_category(v['categories']),
        v['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Venue', 'Latitude', 'Longitude', 'Category', 'Id']
    
    return(nearby_venues)
        

In [56]:
sw = [47.2153657 , 38.9285216]
ne = [47.2153657 , 38.9285216]
venues = searchVenuesInRect(sw=sw, ne=ne)
venues.head()

,Venue,Latitude,Longitude,Category,Id
0,Площадь перед администрацией города,47.215733,38.928230,Plaza,5368f4ad498ea0cb80cef632
1,Администрация Таганрога,47.215517,38.928420,City Hall,4da693d90cb66f658708dafc
2,Театр имени А. П. Чехова,47.216325,38.928217,Theater,4dcbe98a1f6ea1401d49d12a
3,Л'Этуаль,47.215416,38.929266,Cosmetics Shop,4f83002ee4b0b2237e8a6cb1
4,Культ вина,47.215510,38.929310,Wine Bar,5c74142e60255e002c1aefbc


#### Create a map of Taganrog city

In [57]:
# plot venues on map
def plot_venues(venues, loc):
    map_ = folium.Map(location=loc, zoom_start=13)
   
    # add markers to map
    for lat, lng, name, cat in zip(venues['Latitude'], venues['Longitude'], venues['Venue'], venues['Category']):
        label = '{}, {}'.format(name, cat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_)  
        
    return map_

In [58]:
# plot venues on map with a search region
def plotVenuesEx(venues, sw, ne, r=None):
    map_tag = plot_venues(venues, sw)

    # add bounding box or circle to map
    if r == None:
        folium.Rectangle([sw, ne], color='red').add_to(map_tag)
    else:
        folium.Circle(sw, r, color='red').add_to(map_tag)
        
    return map_tag

In [347]:
center = [47.2153657, 38.9285216]
venues = searchVenuesInRect(sw=center, ne=[center[0]+0.005, center[1]+0.005])
#plotVenuesEx(venues=venues, sw=center, ne=[center[0]+0.005, center[1]+0.005])
plot_venues(venues, center)

#### Get the city bounding box

In [59]:
url_bounds = 'https://nominatim.openstreetmap.org/search?format=json&q=Taganrog, Russia&polygon_geojson=1'
# get borders in json format
bounds = requests.get(url_bounds).json()[0]['boundingbox']
# convert to float
city_rect = [float(i) for i in bounds] 

#### Get the list of all venues in the city

In [ ]:
step = 0.005
city_venues = pd.DataFrame(columns=['Venue', 'Latitude', 'Longitude', 'Category', 'Id'])
for lat in np.arange(city_rect[0], city_rect[1], step):
    for lon in np.arange(city_rect[2], city_rect[3], step):
        loc_venues = searchVenuesInRect(sw=[lat, lon], ne=[lat+step, lon+step])
        city_venues = city_venues.append(loc_venues)

print(city_venues.shape)
city_venues.head()

In [472]:
city_venues = city_venues.dropna(subset=['Category'])

In [ ]:
city_venues = city_venues.drop_duplicates('Id')
city_venues = city_venues.reset_index(drop=True)

#### Save the dataset to csv

In [475]:
# save dataframe to csv file
city_venues.to_csv('city_venues.csv', index=False)

#### Draw the coordinate grid

In [31]:
map_tag = folium.Map(location=[47.2153657, 38.9285216], zoom_start=12)

for lat in np.arange(city_rect[0], city_rect[1], step):
    for lon in np.arange(city_rect[2], city_rect[3], step):
        folium.Rectangle([[lat, lon], [lat+step, lon+step]], color='red', weight=0.3).add_to(map_tag)
        
map_tag.fit_bounds([[city_rect[0], city_rect[2]], [city_rect[1], city_rect[3]]])
map_tag

### Get detailed information about each venue

In [118]:
venue_id = '52f6564c498e9e09e3e33147' # ID of Л'Этуаль
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/52f6564c498e9e09e3e33147?client_id=EJSGHSAUO04KW54HYKROI41QUGWWBWGEM1ELH0JCZTTQGHH3&client_secret=4ROYGCW4I2WRJPPRWPFA4JHO33L3SWHNUMCXJU4QXEIEWOPW&v=20180604'

##### Get the venue's details

In [70]:
# Get the venue's overall rating
def rating_value(venue):
    try: return venue['response']['venue']['rating']
    except: return 0

# Get the number of tips
def tips_count(venue):
    try: return venue['response']['venue']['tips']['count']
    except: return 0

# Get the number of likes
def likes_count(venue):
    try: return venue['response']['venue']['likes']['count']
    except: return 0

def api_call_succeeded(json):
    try: return json['meta']['code'] == 200
    except: return False

# get postal code or 347900 if absent
def postal_code(venue):
    try: return venue['response']['venue']['location']['postalCode']
    except: return 347900
    
# get category id of a venue
def category_id(venue):
    try: 
        cat_list = venue['response']['venue']['categories']
    except: 
        return None
    
    if len(cat_list) == 0: 
        return None
    else: 
        return cat_list[0]['id']
    
# the price tier from 1 (least pricey) - 4 (most pricey)
def price_tier(venue):
    try: return venue['response']['venue']['price']['tier']
    except: return 0
    
# Seconds since epoch when the venue was created.
def createdAt(venue):
    try: return (venue['response']['venue']['createdAt'])
    except: return 0    

In [119]:
result = requests.get(url).json()
print('rating:', rating_value(result))
print('tips count:', tips_count(result))
print('likes count:', likes_count(result))
print('postalcode:', postal_code(result))
print('cat_id:', category_id(result))
print('price:', price_tier(result))
print('createdAt:', createdAt(result))
print('Succeeded:', api_call_succeeded(result))

rating: 0
tips count: 0
likes count: 0
succeeded: True
postalcode: 347900
cat_id: 4bf58dd8d48988d109941735
price: 2
createdAt: 1391875660


In [106]:
def get_details(venue_id):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    response = requests.get(url)
    if response.ok:
        result = response.json()
        return [api_call_succeeded(result),
                rating_value(result), 
                likes_count(result), 
                tips_count(result),
                postal_code(result),
                price_tier(result),
                createdAt(result),
                category_id(result)]
    else:
        return [False, None, None, None, None, None, None, None]


### Create additional dataset with venue's details

In [73]:
all_details = pd.DataFrame(columns=['Id', 'processed', 'Rating', 'Likes', 'Tips', 'PostalCode', 'Price_tier', 'createdAt', 'Category_id'])
all_details['Id'] = city_venues['Id']
print(all_details.shape)
all_details.head()

(3525, 9)


,Id,processed,Rating,Likes,Tips,PostalCode,Price_tier,createdAt,Category_id
0,55a0e60c498e873de699ccd1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58401718e9233e42ab37615e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51ecf0f2498edbbbb8c6c29d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53f917c4498e47b1dc40249e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,516166d9e4b0d19a95e6b214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Fill in the dataset with data

In [121]:
import time
for ind in np.arange(all_details.shape[0]):
    if all_details.at[ind, 'processed'] != True:        
        all_details.iloc[ind, 1:] = get_details(all_details.iloc[ind, 0])
        time.sleep(0.3) # delay in seconds
    
all_details

,Id,processed,Rating,Likes,Tips,PostalCode,Price_tier,createdAt,Category_id
0,55a0e60c498e873de699ccd1,True,0,7,0,347900,0,1436608012,4bf58dd8d48988d132951735
1,58401718e9233e42ab37615e,True,5.8,1,0,347900,0,1480595224,52f2ab2ebcbc57f1066b8b46
2,51ecf0f2498edbbbb8c6c29d,True,0,0,0,347900,0,1374482674,4bf58dd8d48988d1e2941735
3,53f917c4498e47b1dc40249e,True,0,1,0,347900,0,1408833476,4bf58dd8d48988d1de941735
4,516166d9e4b0d19a95e6b214,True,0,0,0,347900,0,1365337817,4bf58dd8d48988d12b951735
5,5183984a498e46c7f9c5af0a,True,0,0,1,347900,0,1367578698,4bf58dd8d48988d12b951735
6,4e33e4e41f6efba2411b99e8,True,0,0,0,347900,0,1312023780,4bf58dd8d48988d1e2941735
7,52f6564c498e9e09e3e33147,True,0,0,0,347900,2,1391875660,4bf58dd8d48988d109941735
8,533dc2e4498e008173c0f1e8,True,0,0,0,347900,0,1396556516,4bf58dd8d48988d10a951735
9,533dc2e4498e008173c0f205,True,0,0,0,347900,0,1396556516,4bf58dd8d48988d10a951735


In [122]:
all_details.loc[all_details['processed'] == True]

,Id,processed,Rating,Likes,Tips,PostalCode,Price_tier,createdAt,Category_id
0,55a0e60c498e873de699ccd1,True,0,7,0,347900,0,1436608012,4bf58dd8d48988d132951735
1,58401718e9233e42ab37615e,True,5.8,1,0,347900,0,1480595224,52f2ab2ebcbc57f1066b8b46
2,51ecf0f2498edbbbb8c6c29d,True,0,0,0,347900,0,1374482674,4bf58dd8d48988d1e2941735
3,53f917c4498e47b1dc40249e,True,0,1,0,347900,0,1408833476,4bf58dd8d48988d1de941735
4,516166d9e4b0d19a95e6b214,True,0,0,0,347900,0,1365337817,4bf58dd8d48988d12b951735
5,5183984a498e46c7f9c5af0a,True,0,0,1,347900,0,1367578698,4bf58dd8d48988d12b951735
6,4e33e4e41f6efba2411b99e8,True,0,0,0,347900,0,1312023780,4bf58dd8d48988d1e2941735
7,52f6564c498e9e09e3e33147,True,0,0,0,347900,2,1391875660,4bf58dd8d48988d109941735
8,533dc2e4498e008173c0f1e8,True,0,0,0,347900,0,1396556516,4bf58dd8d48988d10a951735
9,533dc2e4498e008173c0f205,True,0,0,0,347900,0,1396556516,4bf58dd8d48988d10a951735


#### Save details dataframe to csv file

In [110]:
# save dataframe to csv file
all_details.to_csv('city_venues_details.csv', index=False)

### Draw all venues on map

In [48]:
city_venues = pd.read_csv('city_venues.csv')
plot_venues(all_venues)

NameError: name 'plot_venues' is not defined

### Pre-processing

In [355]:
venues_clustering = pd.get_dummies(venues[['Category']], prefix="", prefix_sep="")

# add latitude and longitude columns back to dataframe
venues_clustering[['Latitude', 'Longitude']] = venues[['Latitude', 'Longitude']]

# move latitude and longitude columns to be first
fixed_columns = [venues_clustering.columns[-2], venues_clustering.columns[-1]] + list(venues_clustering.columns[:-2])
venues_clustering = venues_clustering[fixed_columns]

venues_clustering.head()

,Latitude,Longitude,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Dealership,Bakery,Bank,Bar,...,Theater,Theme Park,Toy / Game Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Warehouse,Wine Bar,Wine Shop,Winery,Zoo
0,47.217520,38.929149,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47.215685,38.931146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,47.217143,38.931138,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,47.216197,38.933381,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,47.216325,38.928217,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


#### Normalize the dataset

In [356]:
from sklearn.preprocessing import StandardScaler

X = venues_clustering.values
# X = np.nan_to_num(X)
venues_norm = StandardScaler().fit_transform(X)
venues_norm

array([[ 0.65119305,  0.09232256, -0.07254763, ..., -0.07254763,
        -0.07254763, -0.07254763],
       [-0.41246245,  0.77070172, -0.07254763, ..., -0.07254763,
        -0.07254763, -0.07254763],
       [ 0.43257697,  0.76795734, -0.07254763, ..., -0.07254763,
        -0.07254763, -0.07254763],
       ...,
       [-0.91038484,  0.4877238 , -0.07254763, ..., -0.07254763,
        -0.07254763, -0.07254763],
       [ 0.94469792, -1.27558648, -0.07254763, ..., -0.07254763,
        -0.07254763, -0.07254763],
       [-0.52275722,  0.23528503, -0.07254763, ..., -0.07254763,
        -0.07254763, -0.07254763]])

### Cluster venues
Run k-means to cluster all venues into 5 clusters.

In [357]:
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(venues_norm)
labels = k_means.labels_

print(labels)

[3 3 3 3 3 3 3 3 4 4 4 3 3 3 3 3 3 1 3 3 4 3 3 4 3 4 3 3 3 3 3 3 3 3 4 3 4
 3 4 3 4 4 3 3 4 3 3 0 3 3 3 3 3 3 2 4 3 3 3 4 4 0 3 3 3 3 4 4 3 3 3 4 3 3
 3 3 3 3 3 3 4 3 3 4 3 3 4 3 3 3 4 4 4 4 3 4 4 3 3 3 3 4 3 3 3 3 3 3 4 3 3
 3 3 3 4 4 3 4 3 3 3 4 3 3 4 3 3 3 1 4 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3
 3 3 3 3 3 4 1 4 4 3 3 3 3 1 4 4 3 4 3 3 4 3 3 3 3 3 3 3 4 4 4 3 3 3 3 3 3
 3 3 3 3 4 3]


#### Assigned a label to each venue

In [358]:
venues['Labels'] = labels
venues.head()

,Venue,Latitude,Longitude,Category,Id,Labels
0,"ФГУП ""Федеральный Кадастровый Центр ""Земля""",47.217520,38.929149,Government Building,4e5e11bcd22d7239c19cb7e8,3
1,Три Орешка,47.215685,38.931146,Coffee Shop,53b98963498e9ba407ec3c00,3
2,Альбатрос,47.217143,38.931138,Café,5166e60be4b07cad39a35d31,3
3,"ЗМК ""Кристалл""",47.216197,38.933381,Warehouse,53be56dd498e1c9fd519683c,3
4,Театр имени А. П. Чехова,47.216325,38.928217,Theater,4dcbe98a1f6ea1401d49d12a,3


In [369]:
import matplotlib.cm as cm
import matplotlib.colors as colors

def plot_clusters(venues, center, kclusters):
    # create map
    map_clusters = folium.Map(location=[center[0], center[1]], zoom_start=12)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, cat, cluster in zip(venues['Latitude'], venues['Longitude'], venues['Category'], venues['Labels']):
        label = folium.Popup('Cluster' + str(cluster) + ' - ' + cat, parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
    return map_clusters

In [370]:
plot_clusters(venues, center, 5)

In [436]:
details.at[1, 'processed'] = False